# e) Power & Energy

Calculates power and energy output from the power station for daily river flow data depending on parameters given in a model file.

## Procedure

For each time step (day) in the input river flow data:
1. Determine the required Environmental Flows for each channel
2. Calculate resulting Generation, Canal Flow (and any spill flow)
3. Assign Canal Flow to turbines
4. Calculate headpond and tailwater elevations
5. Calculate headlosses and net head
6. Determine turbine and generator efficiency
7. Calculate power

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| models.csv                  | | List of models with model paramters identified by Model Name  |
| ngonye_flow_daily_ewrx.csv        | Notebook: d_ewr_setup | Daily flow data annotated with Environmental Flow exceedance values |



## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv  | Daily power output  |
| [Model Name]_potential_generation_fdc.csv | Potential generation during testing and commissioning  |


## Libraries and Setup

In [253]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [254]:
year = "2022"

In [255]:
#Papermill parameters

#input_file='CC/ngonye_daily_WC2_56' #'ngonye_daily'#'CC/ngonye_daily_WC1_1'

input_file='ngonye_daily'

In [256]:
input_data='./input_data/'
output_data='./output_data/'

if year != '':
    output_data += year + '/'
input_data, output_data

('./input_data/', './output_data/2022/')

Set the model name below. This is used to lookup model parameters from the models list file.

- Base Case: Base Case
- MW162: Capacity reduced to 162MW
- Headpond50cm: Headpond level increased by 50cm
- EWRAllCs: All river reaches at a C rather than B environmental assurance level
- Extend Head: Minimum head reduce from 10m to 9m
- Tailwater Rise 50cm: Tailwater curve raised by 50cm
- Tailwater Fall 50cm: Tailwater curve dropped by 50cm



In [257]:
model_name='MW270'
dryrun=False #Dont write output files

Read parameters for model

In [258]:
if year != '':
    model_file=input_data + year + "/models.csv"
else:
    model_file=input_data + "models.csv"

models=pd.read_csv(model_file).set_index('ModelName')

model=models.loc[model_name]

out_prefix=model['OutputPrefix'] #'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
tailwater_lift=model['TailwaterLift']#0

plant_capacity=model['Capacity'] #Base 180MW nominal capacity
# units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

model_file, model


('./input_data/2022/models.csv',
 OutputPrefix                                      mw270
 Description                 Capacity increased to 270MW
 EWRCatSet                              Recommendation 1
 TailwaterLift                                       0.0
 HeadpondLift                                        0.0
 Capacity                                            270
 MinimumHead                                          10
 MinimumFlowUnit                                      50
 FixedTailwater                                        0
 UnitsAvailable                                        6
 RatedDischarge                                      250
 RatedHead                                          21.7
 RatedTurbineEfficiency                            0.943
 RatedGeneratorEfficiency                           0.98
 MaximumLoadFactor                                   1.1
 OptimiseTurbines                                      0
 RatedPower                                          Na

## Load Daily Data

Load the daily flow time series which includes the exceedance values used for calculating daily EWRs.

In [259]:
daily=pd.read_csv(output_data + input_file + '.csv')
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
if 'WaterWeek' in daily.columns:
    daily=daily.astype({'WaterWeek': 'int32'})
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35794 entries, 1924-10-01 to 2022-09-30
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LaggedDate           35794 non-null  object 
 1   VicFalls             35794 non-null  float64
 2   Conversion           35794 non-null  float64
 3   Flow                 35794 non-null  float64
 4   Exceedance           35794 non-null  float64
 5   Year                 35794 non-null  int64  
 6   Month                35794 non-null  int64  
 7   Day                  35794 non-null  int64  
 8   MonthId              35794 non-null  float64
 9   WaterYear            35794 non-null  int64  
 10  WaterMonth           35794 non-null  int64  
 11  WaterDay             35794 non-null  int64  
 12  WaterWeek            35794 non-null  int32  
 13  Volume               35794 non-null  float64
 14  Flow_difference      35794 non-null  float64
 15  Flow_difference_pct

## EWR Assurance Category Set
Load the available EWR assurance category sets and select one

In [260]:
category_sets=pd.read_csv(input_data + "ewr_section_categories.csv")
category_sets

,Section,Recommendation 1,All Cs,All Bs,All As
0,A,B,C,B,A
1,C,B,C,B,A
2,D,B,C,B,A
3,E,B,C,B,A
4,FG,C,C,B,A


In [261]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
category_set

,Section,Category
0,A,B
1,C,B
2,D,B
3,E,B
4,FG,C


Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [262]:
ewr_flow_sets=pd.read_csv(input_data + "ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
ewr_flow_sets

Month  Wet  Mod_Wet  Normal  Mod_Dry  Dry
Section Category                                           
A       A            10   75       70      60       47   38
        A            11   73       69      58       45   37
        A            12   77       72      62       48   40
        A             1   83       77      66       52   46
        A             2   90       81      69       56   51
...                 ...  ...      ...     ...      ...  ...
FG      D             5   76       76      66       49   42
        D             6   70       57      46       39   31
        D             7   64       53      41       31   27
        D             8   51       43      32       25   22
        D             9   36       31      23       17   15

[240 rows x 6 columns]

## EWR Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [263]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section'])
ewrs_A=pd.melt(ewrs.loc['A'],id_vars=['Month'], value_vars=['Wet','Mod_Wet','Normal','Mod_Dry','Dry']).set_index(['Month','variable'])
ewrs_C=pd.melt(ewrs.loc['C'],id_vars=['Month'], value_vars=['Wet','Mod_Wet','Normal','Mod_Dry','Dry']).set_index(['Month','variable'])
ewrs_D=pd.melt(ewrs.loc['D'],id_vars=['Month'], value_vars=['Wet','Mod_Wet','Normal','Mod_Dry','Dry']).set_index(['Month','variable'])
ewrs_E=pd.melt(ewrs.loc['E'],id_vars=['Month'], value_vars=['Wet','Mod_Wet','Normal','Mod_Dry','Dry']).set_index(['Month','variable'])
ewrs_FG=pd.melt(ewrs.loc['FG'],id_vars=['Month'], value_vars=['Wet','Mod_Wet','Normal','Mod_Dry','Dry']).set_index(['Month','variable'])

ewrs_A

,,value
Month,variable,
10,Wet,44
11,Wet,42
12,Wet,44
1,Wet,49
2,Wet,55
3,Wet,68
4,Wet,84
5,Wet,71
6,Wet,67


## EWR Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [264]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

In [265]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 
daily[['Flow','EWRBandNo','EWRBandLabel']]

,Flow,EWRBandNo,EWRBandLabel
Date,,,
1924-10-01,138.409487,3,Normal
1924-10-02,138.409487,3,Normal
1924-10-03,138.409487,3,Normal
1924-10-04,138.409487,3,Normal
1924-10-05,138.409487,3,Normal
...,...,...,...
2022-09-26,273.112795,3,Normal
2022-09-27,268.465689,3,Normal
2022-09-28,265.901164,3,Normal


## EWR Flows
Lookup EWR flows

In [266]:
def ewrMonthLookup(frame, month, category):
    return frame.loc[(month,category)].value

In [267]:
#Calculate the required EWRs per channel
daily['EWRChannelA']=daily.apply(lambda x: ewrMonthLookup(ewrs_A,x['Month'],x['EWRBandLabel']),axis=1)
daily['EWRChannelC']=daily.apply(lambda x: ewrMonthLookup(ewrs_C,x['Month'],x['EWRBandLabel']),axis=1)
daily['EWRChannelD']=daily.apply(lambda x: ewrMonthLookup(ewrs_D,x['Month'],x['EWRBandLabel']),axis=1)
daily['EWRChannelE']=daily.apply(lambda x: ewrMonthLookup(ewrs_E,x['Month'],x['EWRBandLabel']),axis=1)
daily['EWRChannelFG']=daily.apply(lambda x: ewrMonthLookup(ewrs_FG,x['Month'],x['EWRBandLabel']),axis=1)
daily[['EWRChannelA','EWRChannelC','EWRChannelD','EWRChannelE','EWRChannelFG']]

,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,EWRChannelFG
Date,,,,,
1924-10-01,37,5,3,2,50
1924-10-02,37,5,3,2,50
1924-10-03,37,5,3,2,50
1924-10-04,37,5,3,2,50
1924-10-05,37,5,3,2,50
...,...,...,...,...,...
2022-09-26,40,6,4,3,65
2022-09-27,40,6,4,3,65
2022-09-28,40,6,4,3,65


In [268]:
#Sum of required EWRs per channel
daily['EWRTotal']=daily['EWRChannelA']+daily['EWRChannelC']+daily['EWRChannelD']+daily['EWRChannelE']+daily['EWRChannelFG']
daily['EWRProportion']=daily['EWRTotal']/daily['Flow']
#If required EWR is greater than total flow then reduce to total flow
daily['EWRTotal']=np.where(daily['EWRTotal']>daily['Flow'],daily['Flow'],daily['EWRTotal'])
daily[['Flow','EWRTotal','EWRProportion','EWRTotal']]

,Flow,EWRTotal,EWRProportion,EWRTotal
Date,,,,
1924-10-01,138.409487,97.0,0.700819,97.0
1924-10-02,138.409487,97.0,0.700819,97.0
1924-10-03,138.409487,97.0,0.700819,97.0
1924-10-04,138.409487,97.0,0.700819,97.0
1924-10-05,138.409487,97.0,0.700819,97.0
...,...,...,...,...
2022-09-26,273.112795,118.0,0.432056,118.0
2022-09-27,268.465689,118.0,0.439535,118.0
2022-09-28,265.901164,118.0,0.443774,118.0


## Canal Flow and Turbine Flow

In [269]:
#Available flow is total flow less EWR flow limited to canal capacity
daily['FlowAvailableForGeneration']=daily['Flow']-daily['EWRTotal']
daily[['Flow','FlowAvailableForGeneration']]

,Flow,FlowAvailableForGeneration
Date,,
1924-10-01,138.409487,41.409487
1924-10-02,138.409487,41.409487
1924-10-03,138.409487,41.409487
1924-10-04,138.409487,41.409487
1924-10-05,138.409487,41.409487
...,...,...
2022-09-26,273.112795,155.112795
2022-09-27,268.465689,150.465689
2022-09-28,265.901164,147.901164


In [270]:
#Canal flow
daily['FlowCanal']=daily.apply(lambda x: min(x['FlowAvailableForGeneration'],units_avialable*max_flow_unit),axis=1)
daily.loc[daily['FlowCanal']<=min_flow_unit,'FlowCanal']=0
daily.loc[daily['FlowCanal']<=min_flow_unit,'LowFlowShutoff']=1
daily[['Flow','FlowCanal','LowFlowShutoff']]

,Flow,FlowCanal,LowFlowShutoff
Date,,,
1924-10-01,138.409487,0.000000,1.0
1924-10-02,138.409487,0.000000,1.0
1924-10-03,138.409487,0.000000,1.0
1924-10-04,138.409487,0.000000,1.0
1924-10-05,138.409487,0.000000,1.0
...,...,...,...
2022-09-26,273.112795,155.112795,NaN
2022-09-27,268.465689,150.465689,NaN
2022-09-28,265.901164,147.901164,NaN


In [274]:
def turbines(day):
    if model['OptimiseTurbines'] == 1:
        if day['flow_canal'] <= rated_flow_unit*units_avialable:
            return math.ceil(day['flow_canal']/rated_flow_unit)
        else:
            return units_avialable
    else:
        return math.ceil(day['FlowCanal'] / max_flow_unit)

In [275]:
#Calculate number of turbines that will operate

daily['Turbines']=daily.apply(lambda x: turbines(x),axis=1)
daily[['Flow','Turbines']]

,Flow,Turbines
Date,,
1924-10-01,138.409487,0
1924-10-02,138.409487,0
1924-10-03,138.409487,0
1924-10-04,138.409487,0
1924-10-05,138.409487,0
...,...,...
2022-09-26,273.112795,1
2022-09-27,268.465689,1
2022-09-28,265.901164,1


In [279]:
if model['OptimiseTurbines'] == 1:
    daily['FlowTurbine1']=daily['FlowCanal']/daily['Turbines']
    daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
    daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
    daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)
else:
    daily['FlowTurbine1']=np.where(daily['Turbines']>=1,daily['FlowCanal']/daily['Turbines'],np.nan)
    daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
    daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
    daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)
daily.loc['2022-03-15'][['FlowTurbine1','FlowTurbine2','FlowTurbine3','FlowTurbine4']]

FlowTurbine1    275.0
FlowTurbine2    275.0
FlowTurbine3    275.0
FlowTurbine4      NaN
Name: 2022-03-15 00:00:00, dtype: object

## Spill and channel flows

In [217]:
#Spill flows are any excess of the required EWR and generation flows
daily['FlowSpill']=daily['Flow']-daily['EWRTotal']-daily['FlowCanal']

#Apportion spill flows in same ratios as EWRs
daily['SpillChannelA']=daily['FlowSpill']*(daily['EWRChannelA']/daily['EWRTotal'])
daily['SpillChannelC']=daily['FlowSpill']*(daily['EWRChannelC']/daily['EWRTotal'])
daily['SpillChannelD']=daily['FlowSpill']*(daily['EWRChannelD']/daily['EWRTotal'])
daily['SpillChannelE']=daily['FlowSpill']*(daily['EWRChannelE']/daily['EWRTotal'])
daily['SpillChannelFG']=daily['FlowSpill']*(daily['EWRChannelFG']/daily['EWRTotal'])
daily[['FlowSpill','SpillChannelA','SpillChannelC','SpillChannelD','SpillChannelE','SpillChannelFG']]

,FlowSpill,SpillChannelA,SpillChannelC,SpillChannelD,SpillChannelE,SpillChannelFG
Date,,,,,,
1924-10-01,41.409487,15.795371,2.13451,1.280706,0.853804,21.345096
1924-10-02,41.409487,15.795371,2.13451,1.280706,0.853804,21.345096
1924-10-03,41.409487,15.795371,2.13451,1.280706,0.853804,21.345096
1924-10-04,41.409487,15.795371,2.13451,1.280706,0.853804,21.345096
1924-10-05,41.409487,15.795371,2.13451,1.280706,0.853804,21.345096
...,...,...,...,...,...,...
2022-09-26,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2022-09-27,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2022-09-28,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [218]:
#Other flows
daily['FlowChannelA']=daily['SpillChannelA']+daily['EWRChannelA']
daily['FlowChannelC']=daily['SpillChannelC']+daily['EWRChannelC']
daily['FlowChannelD']=daily['SpillChannelD']+daily['EWRChannelD']
daily['FlowChannelE']=daily['SpillChannelE']+daily['EWRChannelE']

daily['TmpA']=np.where(daily['FlowChannelA']>100,daily['FlowChannelA']-100,0)
daily['SpillChannelA']=daily['SpillChannelA']-daily['TmpA']
daily['FlowChannelA']=daily['FlowChannelA']-daily['TmpA']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpA']

daily['TmpC']=np.where(daily['FlowChannelC']>18,daily['FlowChannelC']-18,0)
daily['SpillChannelC']=daily['SpillChannelC']-daily['TmpC']
daily['FlowChannelC']=daily['FlowChannelC']-daily['TmpC']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpC']

daily['TmpD']=np.where(daily['FlowChannelD']>25,daily['FlowChannelD']-25,0)
daily['SpillChannelD']=daily['SpillChannelD']-daily['TmpD']
daily['FlowChannelD']=daily['FlowChannelD']-daily['TmpD']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpD']

daily['TmpE']=np.where(daily['FlowChannelE']>80,daily['FlowChannelE']-80,0)
daily['SpillChannelE']=daily['SpillChannelE']-daily['TmpE']
daily['FlowChannelE']=daily['FlowChannelE']-daily['TmpE']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpE']

daily['FlowChannelFG']=daily['SpillChannelFG']+daily['EWRChannelFG']
daily=daily.drop(['TmpA','TmpC','TmpD','TmpE'],axis=1)

daily['FlowLeftChannel']=daily['FlowChannelA']+daily['FlowChannelC']+daily['FlowChannelD']+daily['FlowCanal']

## Levels, headloss and head
Equations from FS Study Report

OLD
$$
\begin{equation}
  OldTailwater Level=\begin{cases}
    -21.429*log_{10}(Q)^2+182.03*log_{10}(Q)+597.15, & \text{if $log_{10}(Q) > 3.8$}.\\
    6.5299*log_{10}(Q)^2-30.155*log_{10}(Q)+999.52, & \text{if $2.3 < log_{10}(Q) < 3.8$}.\\
    -2.6828*log_{10}(Q)^2+11.265*log_{10}(Q)+952.68, & \text{if $log_{10}(Q) < 2.3$}.
  \end{cases}
\end{equation}
$$
OLD

$$
\begin{equation}
  Headpond Level=\begin{cases}
    990.0, & \text{if $Q<5000$}.\\
    0.000081307*Q+989.6, & \text{if $Q<7500$}.\\
    0.000240*Q+988.4, & \text{otherwise}.
  \end{cases}
\end{equation}
$$
$$
\begin{equation}
Headloss Powerhouse=0.00000702Q^2 + 0.00002226Q - 0.00207422
\end{equation}
$$

In [219]:
def headpondLevel(flow):
    if flow<5000:
        return 990.0 + headpond_lift
    elif flow<7500:
        return 0.000081307*flow+989.6 + headpond_lift
    else:
        return 0.000240*flow+988.4 + headpond_lift

In [220]:
tailwater_levels=pd.read_csv(input_data + "tailwater_levels.csv")

In [221]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        return np.interp(flow,tailwater_levels['flow'],tailwater_levels['level']) + tailwater_lift
        #return ((1.0267*10**-11)*flow**3) + ((-3.3603*10**-7)*flow**2) + ((4.2508*10**-3)*flow) + 963.86

        #OLD
        #log_flow=math.log10(flow)
        #if log_flow>=3.8:
        #    return -21.429*log_flow**2+182.03*log_flow+597.15
        #elif log_flow>=2.3:
        #    return 6.5299*log_flow**2-30.155*log_flow+999.52
        #else:
        #    return -2.6828*log_flow**2+11.265*log_flow+952.68

In [222]:
def turbineHeadloss(flow):
    if np.isnan(flow):
        return np.nan
    elif flow==0:
        return 0
    else:
        return 0.00000702*flow**2+0.00002226*flow-0.00207422

In [223]:
daily['LevelTailwater']=daily['Flow'].apply(tailwaterLevel)
daily['LevelHeadpond']=daily['Flow'].apply(headpondLevel)
daily[['Flow','LevelTailwater','LevelHeadpond']]

,Flow,LevelTailwater,LevelHeadpond
Date,,,
1924-10-01,138.409487,964.500000,990.0
1924-10-02,138.409487,964.500000,990.0
1924-10-03,138.409487,964.500000,990.0
1924-10-04,138.409487,964.500000,990.0
1924-10-05,138.409487,964.500000,990.0
...,...,...,...
2022-09-26,273.112795,964.792451,990.0
2022-09-27,268.465689,964.773863,990.0
2022-09-28,265.901164,964.763605,990.0


In [224]:
daily['FlowLeftChannel'].min()

np.float64(64.21058656120107)

Left channel and canal headloss lookup tables

In [225]:
headloss_leftchannel=pd.read_csv(input_data + "headloss_leftchannel.csv")
fHeadlossLeftChannel = interpolate.interp1d(headloss_leftchannel['FlowLeftChannel'], headloss_leftchannel['HeadlossLeftChannel'])
daily['HeadlossLeftChannel']=daily['FlowLeftChannel'].apply(fHeadlossLeftChannel)
daily[['Flow','HeadlossLeftChannel']]

,Flow,HeadlossLeftChannel
Date,,
1924-10-01,138.409487,0.061776
1924-10-02,138.409487,0.061776
1924-10-03,138.409487,0.061776
1924-10-04,138.409487,0.061776
1924-10-05,138.409487,0.061776
...,...,...
2022-09-26,273.112795,0.109811
2022-09-27,268.465689,0.108227
2022-09-28,265.901164,0.107353


In [226]:
headloss_canal=pd.read_csv(input_data + "headloss_canal.csv")
fHeadlossCanal = interpolate.interp1d(headloss_canal['FlowCanal'], headloss_canal['HeadlossCanal'], fill_value="extrapolate")

daily['HeadlossCanal']=daily['FlowCanal'].apply(fHeadlossCanal)
daily['LevelForebay']=daily['LevelHeadpond']-daily['HeadlossLeftChannel']-daily['HeadlossCanal']
daily[['Flow','HeadlossCanal','LevelForebay']]

,Flow,HeadlossCanal,LevelForebay
Date,,,
1924-10-01,138.409487,0.000000,989.938224
1924-10-02,138.409487,0.000000,989.938224
1924-10-03,138.409487,0.000000,989.938224
1924-10-04,138.409487,0.000000,989.938224
1924-10-05,138.409487,0.000000,989.938224
...,...,...,...
2022-09-26,273.112795,0.005511,989.884678
2022-09-27,268.465689,0.005047,989.886726
2022-09-28,265.901164,0.004790,989.887857


In [227]:
daily['HeadlossTurbine1']=daily['FlowTurbine1'].apply(turbineHeadloss)
daily['HeadlossTurbine2']=daily['FlowTurbine2'].apply(turbineHeadloss)
daily['HeadlossTurbine3']=daily['FlowTurbine3'].apply(turbineHeadloss)
daily['HeadlossTurbine4']=daily['FlowTurbine4'].apply(turbineHeadloss)
daily[['Flow','HeadlossTurbine1','HeadlossTurbine2' ,'HeadlossTurbine3','HeadlossTurbine4']]

,Flow,HeadlossTurbine1,HeadlossTurbine2,HeadlossTurbine3,HeadlossTurbine4
Date,,,,,
1924-10-01,138.409487,NaN,NaN,NaN,NaN
1924-10-02,138.409487,NaN,NaN,NaN,NaN
1924-10-03,138.409487,NaN,NaN,NaN,NaN
1924-10-04,138.409487,NaN,NaN,NaN,NaN
1924-10-05,138.409487,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-09-26,273.112795,0.170280,NaN,NaN,NaN
2022-09-27,268.465689,0.160207,NaN,NaN,NaN
2022-09-28,265.901164,0.154779,NaN,NaN,NaN


In [228]:
daily['HeadTurbine1']=np.where(daily['FlowTurbine1']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine1'],np.nan)
daily['HeadTurbine2']=np.where(daily['FlowTurbine2']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine2'],np.nan)
daily['HeadTurbine3']=np.where(daily['FlowTurbine3']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine3'],np.nan)
daily['HeadTurbine4']=np.where(daily['FlowTurbine4']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine4'],np.nan)
daily[['Flow','HeadTurbine1','HeadTurbine2' ,'HeadTurbine3','HeadTurbine4']]

,Flow,HeadTurbine1,HeadTurbine2,HeadTurbine3,HeadTurbine4
Date,,,,,
1924-10-01,138.409487,NaN,NaN,NaN,NaN
1924-10-02,138.409487,NaN,NaN,NaN,NaN
1924-10-03,138.409487,NaN,NaN,NaN,NaN
1924-10-04,138.409487,NaN,NaN,NaN,NaN
1924-10-05,138.409487,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-09-26,273.112795,24.921947,NaN,NaN,NaN
2022-09-27,268.465689,24.952656,NaN,NaN,NaN
2022-09-28,265.901164,24.969474,NaN,NaN,NaN


In [229]:
#Below minimum head then 0 generation. 0 flow in canal.
daily.loc[daily['HeadTurbine1']<=head_minimum,'SpillChannelA']=daily.loc[daily['HeadTurbine1']<=head_minimum,'SpillChannelA']+daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowCanal']
daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowChannelA']=daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowChannelA']+daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowCanal']
daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowSpill']=daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowSpill']+daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowCanal']

daily.loc[daily['HeadTurbine1']<=head_minimum,'FlowCanal']=0
daily.loc[daily['HeadTurbine1']<=head_minimum,'Turbines']=0
daily.loc[daily['HeadTurbine1']<=head_minimum,'LowHeadShutoff']=1


## Turbine and generator efficiency

$$
\begin{equation*}
  Generator Efficiency=\begin{cases}
    98\%, & \text{if $LoadFactor>0.85$}.\\
    97\%, & \text{if $0.45>LoadFactor<0.85$}.\\
    96\%, & \text{if $LoadFactor<0.45$}.
  \end{cases}
\end{equation*}
$$

In [230]:
def generatorEff(load_fact):
    if np.isnan(load_fact):
        return np.nan
    
    if load_fact==0:
        return 0
    elif load_fact<=0.45:
        return 0.96
    elif load_fact<=0.85:
        return 0.97
    else:
        return 0.98

Hill charts from FS Study Reports

In [231]:
def turbineEfficiency2(load_fact,h):
    
    q=load_fact
    
    if np.isnan(h) : return np.nan
    
    if h<=head_minimum: return 0
    
    if q==0: return 0
    elif q<0.214: return 0.77
    elif q<0.22: return 0.78
    elif q<0.234: return 0.79
    elif q<0.248: return 0.80
    elif q<0.256: return 0.81
    elif q<0.266: return 0.82
    elif q<0.28: return 0.83
    elif q<0.288: return 0.84
    elif q<0.298: return 0.85
    elif q<0.33: return 0.86
    elif q<0.366: return 0.87
    elif q<0.402: return 0.88
    elif q<0.4388: return 0.89
    elif q<0.48708: return 0.90
    elif q<0.5872: return 0.91
    elif q<0.6781: return 0.92
    elif q<0.6924:
        if h<24.65: return 0.92
        else: return 0.93
    elif q<0.7992: return 0.93
    elif q<0.8448:
        if h<23: return 0.93
        else: return 0.94
    elif q<0.9882:
        return 0.94
    elif q<1.0:
        if h<24.16: return 0.94
        else: return 0.95
    elif q<1.04:
        if h<23.6: return 0.94
        else: return 0.95
    elif q<1.1:
        if h<22.5: return 0.94
        else: return 0.95
    else:
        return -0.0015*h**2 + 0.0565*h + 0.4

In [232]:
def turbineEfficiency(q_actual,h):
    
    #Scale flow to installed capacity
    q=q_actual*(180/plant_capacity)
    
    if np.isnan(h) or np.isnan(q): return np.nan
    
    if h<=head_minimum: return 0
    
    if q==0: return 0
    elif q<53.5: return 0.77
    elif q<55: return 0.78
    elif q<58.5: return 0.79
    elif q<62: return 0.80
    elif q<64: return 0.81
    elif q<66.5: return 0.82
    elif q<70: return 0.83
    elif q<72: return 0.84
    elif q<74.5: return 0.85
    elif q<82.5: return 0.86
    elif q<91.5: return 0.87
    elif q<100.5: return 0.88
    elif q<109.72: return 0.89
    elif q<121.77: return 0.90
    elif q<146.8: return 0.91
    elif q<169.545: return 0.92
    elif q<173.1:
        if h<24.65: return 0.92
        else: return 0.93
    elif q<199.8: return 0.93
    elif q<211.2:
        if h<23: return 0.93
        else: return 0.94
    elif q<247.05:
        return 0.94
    elif q<250:
        if h<24.16: return 0.94
        else: return 0.95
    elif q<260:
        if h<23.6: return 0.94
        else: return 0.95
    elif q<275:
        if h<22.5: return 0.94
        else: return 0.95
    else:
        return -0.0015*h**2 + 0.0565*h + 0.4

In [233]:
daily['LoadFactorTurbine1']=daily['FlowTurbine1']/rated_flow_unit
daily['LoadFactorTurbine2']=daily['FlowTurbine2']/rated_flow_unit
daily['LoadFactorTurbine3']=daily['FlowTurbine3']/rated_flow_unit
daily['LoadFactorTurbine4']=daily['FlowTurbine4']/rated_flow_unit

In [234]:
daily['EffTurbine1']=daily.apply(lambda x: turbineEfficiency2(x['LoadFactorTurbine1'],x['HeadTurbine1']),axis=1)
daily['EffTurbine2']=daily.apply(lambda x: turbineEfficiency2(x['LoadFactorTurbine2'],x['HeadTurbine2']),axis=1)
daily['EffTurbine3']=daily.apply(lambda x: turbineEfficiency2(x['LoadFactorTurbine3'],x['HeadTurbine3']),axis=1)
daily['EffTurbine4']=daily.apply(lambda x: turbineEfficiency2(x['LoadFactorTurbine4'],x['HeadTurbine4']),axis=1)

In [235]:
daily['EffGen1']=daily['LoadFactorTurbine1'].apply(generatorEff)
daily['EffGen2']=daily['LoadFactorTurbine2'].apply(generatorEff)
daily['EffGen3']=daily['LoadFactorTurbine3'].apply(generatorEff)
daily['EffGen4']=daily['LoadFactorTurbine4'].apply(generatorEff)

## Power and energy

In [236]:
daily['PowerTurbine1']=daily['FlowTurbine1']*daily['HeadTurbine1']*daily['EffTurbine1']*daily['EffGen1']*9.81/1000
daily['PowerTurbine2']=daily['FlowTurbine2']*daily['HeadTurbine2']*daily['EffTurbine2']*daily['EffGen2']*9.81/1000
daily['PowerTurbine3']=daily['FlowTurbine3']*daily['HeadTurbine3']*daily['EffTurbine3']*daily['EffGen3']*9.81/1000
daily['PowerTurbine4']=daily['FlowTurbine4']*daily['HeadTurbine4']*daily['EffTurbine4']*daily['EffGen4']*9.81/1000

daily['Power']=np.nan_to_num(daily['PowerTurbine1'])+np.nan_to_num(daily['PowerTurbine2'])+np.nan_to_num(daily['PowerTurbine3'])+np.nan_to_num(daily['PowerTurbine4'])
daily['Energy']=daily['Power']*24
daily['PerfMwPerCumec']=daily['Power']/daily['Flow']
daily[['Flow','PowerTurbine1','PowerTurbine2','PowerTurbine3','PowerTurbine4','Power', 'Energy','PerfMwPerCumec']]

,Flow,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,
1924-10-01,138.409487,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1924-10-02,138.409487,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1924-10-03,138.409487,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1924-10-04,138.409487,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1924-10-05,138.409487,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
2022-09-26,273.112795,33.842166,NaN,NaN,NaN,33.842166,812.211991,0.123913
2022-09-27,268.465689,32.868723,NaN,NaN,NaN,32.868723,788.849354,0.122432
2022-09-28,265.901164,32.330286,NaN,NaN,NaN,32.330286,775.926869,0.121588


## Generation potential for testing and commissioning

In [237]:
#Calculate 'potential generation' aginst the flow FDC.
#Potential generation assumes flow can be stored in the headpond and only head is limiting

fdc=pd.read_csv("./output_data/ngonye_flow_fdc.csv").set_index('Exceedance')
fdc['LevelTailwater']=fdc['Flow'].apply(tailwaterLevel)
fdc['LevelHeadpond']=fdc['Flow'].apply(headpondLevel)

fdc['1uFlowLeftChannel']=max_flow_unit+100
fdc['1uHeadlossLeftChannel']=fdc['1uFlowLeftChannel'].apply(fHeadlossLeftChannel)
fdc['2uFlowLeftChannel']=(max_flow_unit*2)+100
fdc['2uHeadlossLeftChannel']=fdc['2uFlowLeftChannel'].apply(fHeadlossLeftChannel)
fdc['3uFlowLeftChannel']=(max_flow_unit*3)+100
fdc['3uHeadlossLeftChannel']=fdc['3uFlowLeftChannel'].apply(fHeadlossLeftChannel)
fdc['4uFlowLeftChannel']=(max_flow_unit*4)+100
fdc['4uHeadlossLeftChannel']=fdc['4uFlowLeftChannel'].apply(fHeadlossLeftChannel)

u1headlossCanal=fHeadlossCanal(max_flow_unit)
u2headlossCanal=fHeadlossCanal(max_flow_unit*2)
u3headlossCanal=fHeadlossCanal(max_flow_unit*3)
u4headlossCanal=fHeadlossCanal(max_flow_unit*4)

fdc['1uLevelForebay']=fdc['LevelHeadpond']-fdc['1uHeadlossLeftChannel']-u1headlossCanal
fdc['2uLevelForebay']=fdc['LevelHeadpond']-fdc['2uHeadlossLeftChannel']-u2headlossCanal
fdc['3uLevelForebay']=fdc['LevelHeadpond']-fdc['3uHeadlossLeftChannel']-u3headlossCanal
fdc['4uLevelForebay']=fdc['LevelHeadpond']-fdc['4uHeadlossLeftChannel']-u4headlossCanal

headlossTurbine=turbineHeadloss(max_flow_unit)

fdc['1uHead']=fdc['1uLevelForebay']-fdc['LevelTailwater']-headlossTurbine
fdc['2uHead']=fdc['2uLevelForebay']-fdc['LevelTailwater']-headlossTurbine
fdc['3uHead']=fdc['3uLevelForebay']-fdc['LevelTailwater']-headlossTurbine
fdc['4uHead']=fdc['4uLevelForebay']-fdc['LevelTailwater']-headlossTurbine

loadFactor=max_flow_unit/rated_flow_unit
fdc['1uEffTurbine']=fdc.apply(lambda x: turbineEfficiency2(loadFactor,x['1uHead']),axis=1)
fdc['2uEffTurbine']=fdc.apply(lambda x: turbineEfficiency2(loadFactor,x['2uHead']),axis=1)
fdc['3uEffTurbine']=fdc.apply(lambda x: turbineEfficiency2(loadFactor,x['3uHead']),axis=1)
fdc['4uEffTurbine']=fdc.apply(lambda x: turbineEfficiency2(loadFactor,x['4uHead']),axis=1)

effGen=generatorEff(loadFactor)

fdc['1uPowerPotential']=1*max_flow_unit*fdc['1uHead']*fdc['1uEffTurbine']*effGen*9.81/1000
fdc['2uPowerPotential']=2*max_flow_unit*fdc['2uHead']*fdc['2uEffTurbine']*effGen*9.81/1000
fdc['3uPowerPotential']=3*max_flow_unit*fdc['3uHead']*fdc['3uEffTurbine']*effGen*9.81/1000
fdc['4uPowerPotential']=4*max_flow_unit*fdc['4uHead']*fdc['4uEffTurbine']*effGen*9.81/1000

fdc

,Flow,LevelTailwater,LevelHeadpond,1uFlowLeftChannel,1uHeadlossLeftChannel,2uFlowLeftChannel,2uHeadlossLeftChannel,3uFlowLeftChannel,3uHeadlossLeftChannel,4uFlowLeftChannel,...,3uHead,4uHead,1uEffTurbine,2uEffTurbine,3uEffTurbine,4uEffTurbine,1uPowerPotential,2uPowerPotential,3uPowerPotential,4uPowerPotential
Exceedance,,,,,,,,,,,,,,,,,,,,,
0.000,9912.101075,982.443745,990.778904,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,7.089189,6.479194,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
0.001,8829.697418,981.756150,990.519127,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,7.517007,6.907012,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
0.002,7890.806381,981.127358,990.293794,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,7.920465,7.310470,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
0.003,7005.060692,980.294757,990.169560,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,8.628833,8.018838,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
0.004,6479.994666,979.645193,990.126869,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,9.235705,8.625710,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.996,142.406007,964.500000,990.000000,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,24.254029,23.644034,0.879112,0.882478,0.887966,0.897327,57.578535,114.70216,170.816235,224.36767
0.997,130.554212,964.500000,990.000000,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,24.254029,23.644034,0.879112,0.882478,0.887966,0.897327,57.578535,114.70216,170.816235,224.36767
0.998,124.486097,964.500000,990.000000,375.0,0.166494,650.0,0.293478,925.0,0.501036,1200.0,...,24.254029,23.644034,0.879112,0.882478,0.887966,0.897327,57.578535,114.70216,170.816235,224.36767


In [238]:
fPotential1u = interpolate.interp1d(fdc['Flow'], fdc['1uPowerPotential'])
fPotential2u = interpolate.interp1d(fdc['Flow'], fdc['2uPowerPotential'])
fPotential3u = interpolate.interp1d(fdc['Flow'], fdc['3uPowerPotential'])
fPotential4u = interpolate.interp1d(fdc['Flow'], fdc['4uPowerPotential'])

In [239]:
daily['PotentialPower1u']=daily['Flow'].apply(fPotential1u)
daily['PotentialPower2u']=daily['Flow'].apply(fPotential2u)
daily['PotentialPower3u']=daily['Flow'].apply(fPotential3u)
daily['PotentialPower4u']=daily['Flow'].apply(fPotential4u)

## Save

In [240]:
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.384095,138.409487,0.998,1924,10,1,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.578535,114.702160,170.816235,224.367670
1924-10-02,1924-10-13,100.0,1.384095,138.409487,0.998,1924,10,2,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.578535,114.702160,170.816235,224.367670
1924-10-03,1924-10-14,100.0,1.384095,138.409487,0.998,1924,10,3,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.578535,114.702160,170.816235,224.367670
1924-10-04,1924-10-15,100.0,1.384095,138.409487,0.998,1924,10,4,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.578535,114.702160,170.816235,224.367670
1924-10-05,1924-10-16,100.0,1.384095,138.409487,0.998,1924,10,5,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.578535,114.702160,170.816235,224.367670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,2022-10-07,293.0,0.932126,273.112795,0.825,2022,9,26,2022.09,2021,...,NaN,NaN,NaN,33.842166,812.211991,0.123913,57.227834,113.968760,169.636020,222.603085
2022-09-27,2022-10-08,289.0,0.928947,268.465689,0.839,2022,9,27,2022.09,2021,...,NaN,NaN,NaN,32.868723,788.849354,0.122432,57.250850,114.016802,169.713117,222.717871
2022-09-28,2022-10-09,286.0,0.929724,265.901164,0.847,2022,9,28,2022.09,2021,...,NaN,NaN,NaN,32.330286,775.926869,0.121588,57.263512,114.043238,169.755551,222.781075


In [241]:
output_data + out_prefix + '_pe_daily.csv'

'./output_data/2022/scenarios/mw270_pe_daily.csv'

In [242]:
if dryrun==False:
    daily.to_csv(output_data + out_prefix + '_pe_daily.csv')
    fdc.to_csv(output_data + out_prefix + '_potential_generation_fdc.csv')